# สร้างตัวแทน AI ที่มีหน่วยความจำถาวรด้วย Mem0, Semantic Kernel และ Azure AI Search

โน้ตบุ๊กนี้แสดงวิธีการสร้างตัวแทนการจองการเดินทางที่ชาญฉลาดซึ่งสามารถจดจำความชอบของผู้ใช้ในบทสนทนาต่าง ๆ โดยการรวม Mem0, Semantic Kernel และ Azure AI Search เราสร้างตัวแทนที่ให้คำแนะนำการเดินทางที่ปรับแต่งตามปฏิสัมพันธ์ในอดีต

## สิ่งที่คุณจะได้เรียนรู้:
1. **การรวม Mem0**: วิธีใช้ Mem0 เป็นชั้นหน่วยความจำสำหรับตัวแทน AI
2. **Azure AI Search เป็น Vector Store**: เก็บและดึงข้อมูลหน่วยความจำโดยใช้การค้นหาเชิงความหมาย
3. **ความชอบของผู้ใช้ที่ถาวร**: จดจำความชอบของผู้ใช้ในเซสชันแชทต่าง ๆ
4. **ปลั๊กอิน Semantic Kernel**: สร้างปลั๊กอินที่ใช้ประโยชน์จากทั้งหน่วยความจำและความสามารถในการค้นหา

## สิ่งที่ต้องเตรียม:
- การตั้งค่าการปรับใช้ Azure OpenAI
- การสร้างบริการ Azure AI Search
- ความเข้าใจพื้นฐานเกี่ยวกับแนวคิด Semantic Kernel


## การทำความเข้าใจสถาปัตยกรรมหน่วยความจำ

### Mem0 คืออะไร?

**Mem0** เป็นชั้นหน่วยความจำอัจฉริยะที่มีคุณสมบัติดังนี้:
- **หน่วยความจำระยะยาว**: เก็บข้อมูลเกี่ยวกับความชอบของผู้ใช้, การโต้ตอบในอดีต, และข้อมูลที่เรียนรู้
- **การค้นหาเชิงความหมาย**: ดึงข้อมูลที่เกี่ยวข้องตามบริบท
- **พื้นที่จัดเก็บเฉพาะผู้ใช้**: รักษาพื้นที่หน่วยความจำแยกสำหรับผู้ใช้แต่ละคน
- **ความเกี่ยวข้องอัตโนมัติ**: แสดงหน่วยความจำที่เกี่ยวข้องมากที่สุดสำหรับบริบทปัจจุบัน

### วิธีการทำงานร่วมกันของส่วนประกอบ:
```
┌─────────────────┐     ┌──────────────────┐     ┌─────────────────┐
│  Semantic       │────▶│      Mem0        │────▶│  Azure AI       │
│  Kernel Agent   │     │  Memory Layer    │     │  Search         │
└─────────────────┘     └──────────────────┘     └─────────────────┘
         │                       │                         │
         │                       │                         │
    Processes              Stores/Retrieves          Vector Store
    User Input             User Preferences         for Memories &
                          & Context                  Travel Data
```


In [ ]:
! pip install mem0ai

## นำเข้าชุดคำสั่งที่จำเป็น


In [ ]:
import json
import os
from typing import Annotated, List, Dict, Any
from datetime import datetime
import uuid

from IPython.display import display, HTML, Markdown
from dotenv import load_dotenv

# Azure AI Search
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndex,
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    SearchField,
    VectorSearchAlgorithmMetric
)

# Mem0
from mem0 import Memory

# Semantic Kernel
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.functions import kernel_function
from semantic_kernel.contents import ChatHistory
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread

## การกำหนดค่าการตั้งค่า


In [ ]:
# Load environment variables
load_dotenv()

# Azure OpenAI Configuration
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")  # Use a recent API version


# Azure AI Search Configuration
search_service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
search_api_key = os.getenv("AZURE_SEARCH_API_KEY")

# Index names
travel_index_name = "travel-hotels"
memory_index_name = "mem0-memories"



## เริ่มต้นใช้งาน Azure AI Search สำหรับข้อมูลการท่องเที่ยว

ก่อนอื่น เราจะตั้งค่า Azure AI Search พร้อมข้อมูลตัวอย่างของโรงแรมและสถานที่ท่องเที่ยวที่ตัวแทนของเราสามารถค้นหาได้


In [ ]:
# Initialize search clients
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=AzureKeyCredential(search_api_key)
)

# Create travel data index if it doesn't exist
travel_fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchableField(name="name", type=SearchFieldDataType.String),
    SearchableField(name="description", type=SearchFieldDataType.String),
    SearchableField(name="location", type=SearchFieldDataType.String),
    SearchableField(name="amenities", type=SearchFieldDataType.String),
    SimpleField(name="price_per_night", type=SearchFieldDataType.Double),
    SimpleField(name="rating", type=SearchFieldDataType.Double),
    SearchableField(name="tags", type=SearchFieldDataType.String, collection=True)
]

travel_index = SearchIndex(name=travel_index_name, fields=travel_fields)

try:
    index_client.get_index(travel_index_name)
    print(f"✅ Index '{travel_index_name}' already exists")
except:
    index_client.create_index(travel_index)
    print(f"✅ Created index '{travel_index_name}'")

# Initialize search client for travel data
travel_search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=travel_index_name,
    credential=AzureKeyCredential(search_api_key)
)

In [ ]:
# Add sample travel data
sample_hotels = [
    {
        "id": "1",
        "name": "Le Meurice Paris",
        "description": "Luxury palace hotel with Michelin-starred dining and views of the Tuileries Garden",
        "location": "Paris, France",
        "amenities": "Spa, Michelin Restaurant, Concierge, Room Service, Fitness Center",
        "price_per_night": 850,
        "rating": 4.8,
        "tags": ["luxury", "romantic", "historic", "fine-dining", "spa"]
    },
    {
        "id": "2",
        "name": "Four Seasons Maui",
        "description": "Beachfront resort with world-class spa and family-friendly activities",
        "location": "Maui, Hawaii",
        "amenities": "Beach Access, Kids Club, Multiple Pools, Spa, Golf Course",
        "price_per_night": 695,
        "rating": 4.7,
        "tags": ["beach", "family-friendly", "resort", "spa", "golf"]
    },
    {
        "id": "3",
        "name": "Aman Tokyo",
        "description": "Minimalist luxury hotel with panoramic city views and traditional onsen",
        "location": "Tokyo, Japan",
        "amenities": "Onsen, City Views, Fine Dining, Spa, Business Center",
        "price_per_night": 780,
        "rating": 4.9,
        "tags": ["luxury", "business", "spa", "city", "minimalist"]
    },
    {
        "id": "4",
        "name": "Hotel Sacher Vienna",
        "description": "Historic hotel home of the original Sachertorte with elegant rooms",
        "location": "Vienna, Austria",
        "amenities": "Historic Cafe, Concierge, Accessible Rooms, Pet-Friendly",
        "price_per_night": 420,
        "rating": 4.6,
        "tags": ["historic", "accessible", "pet-friendly", "cultural", "cafe"]
    },
    {
        "id": "5",
        "name": "Fairmont Whistler",
        "description": "Ski-in/ski-out resort with family suites and mountain views",
        "location": "Whistler, Canada",
        "amenities": "Ski Access, Family Suites, Heated Pool, Kids Programs",
        "price_per_night": 380,
        "rating": 4.5,
        "tags": ["ski", "family-friendly", "mountain", "resort", "accessible"]
    }
]

# Upload hotels to search index
travel_search_client.upload_documents(documents=sample_hotels)
print(f"✅ Uploaded {len(sample_hotels)} hotels to search index")

## เริ่มต้น Mem0 ด้วย Azure AI Search

ตอนนี้เราจะตั้งค่า Mem0 ให้ใช้ Azure AI Search เป็นที่เก็บเวกเตอร์สำหรับหน่วยความจำถาวร


In [ ]:
mem0_config = {
    "llm": {
        "provider": "azure_openai",
        "config": {
            "model": azure_openai_deployment,
            "temperature": 0.2,
            "max_tokens": 1500,
            "azure_kwargs": {
                "azure_deployment": azure_openai_deployment,
                "api_version": api_version,
                "azure_endpoint": azure_openai_endpoint,
                "api_key": azure_openai_api_key,
            }
        }
    },
    "vector_store": {
        "provider": "azure_ai_search",
        "config": {
            "service_name": search_service_endpoint.split("//")[1].split(".")[0],
            "api_key": search_api_key,
            "collection_name": "mem0",
            "embedding_model_dims": 1536
        }
    },
    "embedder": {
        "provider": "azure_openai",
        "config": {
            "model": "text-embedding-ada-002",  # Your embedding deployment name
            "azure_kwargs": {
                "azure_deployment": "text-embedding-ada-002",  # Update if different
                "api_version": api_version,
                "azure_endpoint": azure_openai_endpoint,
                "api_key": azure_openai_api_key,
            }
        }
    }
}

# Initialize Mem0
memory = Memory.from_config(mem0_config)
# Test the memory system
print("🧪 Testing Mem0 setup...")
test_messages = [
    {"role": "user", "content": "I prefer luxury hotels with spa services."},
    {"role": "assistant", "content": "I'll remember you prefer luxury hotels with spa services for future recommendations."}
]
memory.add(test_messages, user_id="test_user",
           metadata={"category": "preferences"})
test_memories = memory.get_all(user_id="test_user")
print(f"✅ Mem0 test successful! Found {len(test_memories)} memories")

## สร้างปลั๊กอินการจองการเดินทาง

ปลั๊กอินนี้มีฟังก์ชันสำหรับค้นหาโรงแรมและจัดการการตั้งค่าของผู้ใช้ผ่าน Mem0.


In [ ]:
class TravelBookingPlugin:
    """Plugin for searching hotels and managing user travel preferences"""

    def __init__(self, search_client: SearchClient, memory: Memory):
        self.search_client = search_client
        self.memory = memory

    @kernel_function(
        description="Search for hotels based on criteria like location, amenities, or tags"
    )
    def search_hotels(
        self,
        query: Annotated[str, "Search query for hotels (location, amenities, etc.)"],
        max_results: Annotated[int, "Maximum number of results to return"] = 3
    ) -> Annotated[str, "List of hotels matching the search criteria"]:
        """Search for hotels in the travel database"""
        results = self.search_client.search(
            search_text=query,
            top=max_results,
            include_total_count=True
        )

        hotels = []
        for result in results:
            hotels.append({
                "name": result["name"],
                "location": result["location"],
                "description": result["description"],
                "price_per_night": result["price_per_night"],
                "rating": result["rating"],
                "amenities": result["amenities"],
                "tags": result["tags"]
            })

        return json.dumps(hotels, indent=2)

    @kernel_function(
        description="Store user travel preferences and important information in memory"
    )
    def store_user_preference(
        self,
        user_id: Annotated[str, "User identifier"],
        preference: Annotated[str,
                              "User preference or information to remember"]
    ) -> Annotated[str, "Confirmation of stored preference"]:
        """Store user preferences in Mem0 memory"""
        print(f"DEBUG: Storing preference for {user_id}: {preference}")

        try:
            # Simply add the preference to memory
            self.memory.add(preference, user_id=user_id)
            return f"✅ Stored: {preference}"
        except Exception as e:
            return f"❌ Error storing preference: {str(e)}"
        
    @kernel_function(
        description="Get all stored preferences for a user"
    )
    def get_user_preferences(
        self,
        user_id: Annotated[str, "User identifier"]
    ) -> Annotated[str, "All user preferences and memories"]:
        """Get all memories for a specific user"""
        print(f"DEBUG: Getting all preferences for {user_id}")

        try:
            # Get all memories for the user
            results = self.memory.get_all(user_id=user_id)

            # Handle the dict response with 'results' key
            if isinstance(results, dict) and 'results' in results:
                results = results.get('results', [])

            if not results:
                return f"No preferences found for user {user_id}"

            # Format results
            memories = []
            for result in results:
                if isinstance(result, dict):
                    memory_text = result.get('memory', str(result))
                    memories.append(memory_text)
                else:
                    memories.append(str(result))

            return f"User preferences for {user_id}:\n- " + "\n- ".join(memories)

        except Exception as e:
            print(f"ERROR getting preferences: {str(e)}")
            return f"No preferences found for user {user_id}"


    @kernel_function(
        description="Search user's memories for relevant information"
    )
    def search_memories(
        self,
        user_id: Annotated[str, "User identifier"],
        query: Annotated[str,
                         "What to search for (e.g., 'family vacation', 'dietary restrictions')"]
    ) -> Annotated[str, "Relevant memories"]:
        """Search user memories using Mem0"""
        print(f"DEBUG: Searching memories for {user_id} with query: '{query}'")

        try:
            # Let Mem0 handle the search and ranking
            results = self.memory.search(query, user_id=user_id)

            # Handle the dict response with 'results' key
            if isinstance(results, dict) and 'results' in results:
                results = results.get('results', [])

            if not results:
                return f"No memories found for query: {query}"

            # Format results
            memories = []
            for result in results:
                if isinstance(result, dict):
                    memory_text = result.get('memory', str(result))
                    # Include relevance score if available
                    score = result.get('score', None)
                    if score:
                        memories.append(
                            f"{memory_text} (relevance: {score:.2f})")
                    else:
                        memories.append(memory_text)
                else:
                    memories.append(str(result))

            return "Relevant memories:\n- " + "\n- ".join(memories)

        except Exception as e:
            print(f"ERROR: {str(e)}")
            return "No memories found."
        
        
    

## เริ่มต้น Semantic Kernel Agent

สร้างตัวแทนการจองการเดินทางของเราพร้อมการเข้าถึงปลั๊กอินการจองการเดินทาง


In [ ]:
# Create the kernel
kernel = Kernel()

# Add Azure OpenAI service
chat_service = AzureChatCompletion(
    deployment_name=azure_openai_deployment,
    endpoint=azure_openai_endpoint,
    api_key=azure_openai_api_key,
)
kernel.add_service(chat_service)

# Create and add the travel booking plugin
travel_plugin = TravelBookingPlugin(travel_search_client, memory)
kernel.add_plugin(
    plugin_name="TravelBooking",
    plugin=travel_plugin
)

# Create the travel agent
# Create the travel agent
travel_agent = ChatCompletionAgent(
    service=chat_service,
    name="TravelBookingAssistant",
    instructions="""
    You are a personalized travel booking assistant with memory.
    
    WORKFLOW:
    1. When a user asks for help, search their memories using search_memories() with a relevant query
    2. Use the memories to personalize your response
    3. Store any new preferences they mention using store_user_preference()
    4. When the users is booking a new trip, first retrieve the users general travel preferences of the user by creating queries for hotels, dietary restrictions, location, amenities and budget. THEN use search_hotels() to find suitable options.
    5. Do not recommend hotels that are over budget. 
    
    IMPORTANT: For ALL memory operations (search_memories and store_user_preference), 
    you MUST use user_id='sarah_johnson_123' exactly as written.

    Example queries:
    - User asks about booking a trip → search_memories(query="preferences")
    - User asks about booking a trip → search_memories(query="dietary restrictions")
    - User asks about booking a trip → search_memories(query="location")
    - User asks about booking a trip → search_memories(query="amenities")
    - User asks about booking a trip → search_memories(query="budget")

    Always acknowledge what you found in their memories when responding.""",
    plugins=[travel_plugin]
)

## ฟังก์ชันช่วยสำหรับการแสดงผลที่สะอาด


In [ ]:
def display_message(role: str, content: str, color: str = "#2E8B57", emoji: str = ""):
    """Display a message with nice formatting"""
    html = f"""
    <div style='
        margin: 10px 0; 
        padding: 15px 20px; 
        border-left: 4px solid {color}; 
        background: rgba(128, 128, 128, 0.05); 
        border-radius: 8px;
    '>
        <strong style='color: {color}; font-size: 16px;'>{emoji} {role}:</strong><br>
        <div style='margin-top: 10px; white-space: pre-wrap; font-size: 14px; line-height: 1.6;'>{content}</div>
    </div>
    """
    display(HTML(html))

def display_memory_operation(operation: str, details: str, color: str = "#9370DB"):
    """Display memory operations for educational purposes"""
    html = f"""
    <div style='
        margin: 5px 20px;
        padding: 10px 15px;
        background: rgba(147, 112, 219, 0.1);
        border: 1px solid {color};
        border-radius: 6px;
        font-family: monospace;
        font-size: 13px;
    '>
        <strong style='color: {color};'>🧠 Memory {operation}:</strong>
        <div style='margin-top: 5px; color: #555;'>{details}</div>
    </div>
    """
    display(HTML(html))

def display_function_call(function_name: str, args: dict, result: str = None):
    """Display function calls for transparency"""
    html = f"""
    <details style='margin: 5px 20px; padding: 10px; background: rgba(0, 123, 255, 0.05); border: 1px solid #007BFF; border-radius: 6px;'>
        <summary style='cursor: pointer; font-weight: bold; color: #007BFF;'>⚙️ Function Call: {function_name}</summary>
        <div style='margin-top: 10px; font-family: monospace; font-size: 12px;'>
            <div><strong>Arguments:</strong> {json.dumps(args, indent=2)}</div>
    """
    if result:
        html += f"<div style='margin-top: 10px;'><strong>Result:</strong><pre style='background: #f8f8f8; padding: 8px; border-radius: 4px; overflow-x: auto;'>{result}</pre></div>"
    html += "</div></details>"
    display(HTML(html))

## สาธิตการจองการเดินทางพร้อมความจำ

มาลองดูสถานการณ์การจองการเดินทางที่สมจริง ซึ่งแสดงให้เห็นว่าเอเจนต์จดจำและใช้ความชอบของผู้ใช้ได้อย่างไร


### สถานการณ์ที่ 1: ผู้ใช้ครั้งแรก - วางแผนทริปวันครบรอบ


In [ ]:
# User ID for our demonstration
sarah_user_id = "sarah_johnson_123"

print("🎯 SCENARIO 1: Sarah's First Booking - Anniversary Trip\n")

# Create a new chat history for Sarah
sarah_chat = ChatHistoryAgentThread()

# First conversation
user_message1 = """Hi! I'm Sarah and I'm planning a special trip for my 10th wedding anniversary. 
We love romantic destinations, fine dining, and spa experiences. My husband has mobility issues, 
so we need accessible accommodations. Our budget is around $700-800 per night."""

display_message("Sarah", user_message1, "#4fc3f7", "👤")

# Extract and display function calls for educational purposes
# Process with agent
response_content = ""
function_calls_made = []

async for response in travel_agent.invoke(
    messages=user_message1,
    thread=sarah_chat
):
    if response.content:
        response_content = str(response.content)

    # Parse function calls from the thread
    if hasattr(response, 'thread'):
        sarah_thread = response.thread

        # Check for function calls in the latest messages
        async for msg in sarah_thread.get_messages():
            if hasattr(msg, 'items') and msg.items:
                for item in msg.items:
                    if hasattr(item, 'function_invoke_attempt') and item.function_invoke_attempt:
                        func_call = item.function_invoke_attempt
                        function_info = {
                            'name': func_call.function_name,
                            'arguments': func_call.arguments,
                            'result': item.function_result.value if hasattr(item, 'function_result') else None
                        }
                        if function_info not in function_calls_made:
                            function_calls_made.append(function_info)

                            # Display the actual function calls
                            if 'get_user_preferences' in func_call.function_name:
                                display_memory_operation(
                                    "Retrieval", f"Checking existing preferences for user: {func_call.arguments.get('user_id', sarah_user_id)}")
                            elif 'store_user_preference' in func_call.function_name:
                                display_memory_operation(
                                    "Storage", f"Storing: {func_call.arguments.get('preference', '')}")
                            elif 'search_hotels' in func_call.function_name:
                                display_function_call(
                                    func_call.function_name,
                                    func_call.arguments,
                                    item.function_result.value if hasattr(
                                        item, 'function_result') else None
                                )

display_message("Travel Assistant", response_content, "#81c784", "🤖")

In [ ]:
user_message2 = """The Hotel Sacher sounds perfect! We're both vegetarian and I have a severe nut allergy. 
Can you tell me more about their dining options?"""

display_message("Sarah", user_message2, "#4fc3f7", "👤")

response2_content = ""
async for response in travel_agent.invoke(
    messages=user_message2,
    thread=sarah_thread
):
    if response.content:
        response2_content = str(response.content)

    if hasattr(response, 'thread'):
        sarah_thread = response.thread

        # Parse new function calls
        async for msg in sarah_thread.get_messages():
            if hasattr(msg, 'items') and msg.items:
                for item in msg.items:
                    if hasattr(item, 'function_invoke_attempt') and item.function_invoke_attempt:
                        func_call = item.function_invoke_attempt
                        if 'store_user_preference' in func_call.function_name and func_call.arguments.get('preference'):
                            pref = func_call.arguments.get('preference', '')
                            if 'vegetarian' in pref.lower() or 'nut allergy' in pref.lower():
                                display_memory_operation(
                                    "Storage", f"Storing: {pref}")

display_message("Travel Assistant", response2_content, "#81c784", "🤖")

In [ ]:

# After running all scenarios, verify memories are stored
from azure.search.documents import SearchClient
print("\n\n🔍 VERIFYING MEM0 STORAGE\n")

# Check Azure AI Search directly
mem0_search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name="mem0",
    credential=AzureKeyCredential(search_api_key)
)

try:
    # Count documents in the index
    results = mem0_search_client.search(
        search_text="*", include_total_count=True)
    total_docs = results.get_count()
    print(f"📊 Total documents in Mem0 index: {total_docs}")

    # Show first few documents
    print("\nSample documents:")
    for i, doc in enumerate(results):
        if i < 3:  # Show first 3
            print(f"\nDocument {i+1}:")
            print(f"  ID: {doc.get('id', 'N/A')}")
            print(f"  User ID: {doc.get('user_id', 'N/A')}")
            print(f"  Memory: {doc.get('payload', 'N/A')}")
except Exception as e:
    print(f"❌ Error checking Mem0 index: {str(e)}")
    print("The index might not be created yet or might be empty.")

In [ ]:
# Enhanced verification to debug Mem0 responses
print("🔍 ENHANCED MEM0 VERIFICATION\n")

# Create a unique test user
test_user = f"debug_user_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
test_memory = "I am vegetarian with a peanut allergy and love beach destinations"

print(f"1. Adding memory for {test_user}...")
add_result = memory.add(test_memory, user_id=test_user)
print(f"   Raw add result: {add_result}")

# Check if the result has a 'results' key
if isinstance(add_result, dict) and 'results' in add_result:
    actual_results = add_result.get('results', [])
    print(
        f"   Actual memories added: {len(actual_results) if isinstance(actual_results, list) else 0}")
    if isinstance(actual_results, list):
        for mem in actual_results:
            print(
                f"   - ID: {mem.get('id', 'N/A')}, Memory: {mem.get('memory', 'N/A')}")

print("\n2. Testing get_all()...")
all_mems = memory.get_all(user_id=test_user)
print(f"   Raw response: {all_mems}")
print(f"   Response type: {type(all_mems)}")

# Check if it's a dict with 'results' key
if isinstance(all_mems, dict):
    print(f"   Dict keys: {list(all_mems.keys())}")
    if 'results' in all_mems:
        results_value = all_mems['results']
        print(f"   'results' value type: {type(results_value)}")
        print(f"   'results' value: {results_value}")

        # If results is actually a list, show the memories
        if isinstance(results_value, list):
            print(f"   Number of memories: {len(results_value)}")
            for i, mem in enumerate(results_value):
                print(f"   Memory {i}: {mem}")

print("\n3. Testing search()...")
search_results = memory.search("peanut allergy", user_id=test_user)
print(f"   Raw response: {search_results}")
print(f"   Response type: {type(search_results)}")

# Check if it's a dict with 'results' key
if isinstance(search_results, dict):
    print(f"   Dict keys: {list(search_results.keys())}")
    if 'results' in search_results:
        results_value = search_results['results']
        print(f"   'results' value type: {type(results_value)}")
        print(f"   'results' value: {results_value}")

print("\n4. Testing direct API access...")
# Try to access memories through Azure AI Search directly
try:
    mem0_search_client = SearchClient(
        endpoint=search_service_endpoint,
        index_name="mem0",
        credential=AzureKeyCredential(search_api_key)
    )

    # Wait a moment for indexing
    import time
    time.sleep(2)

    # Search for the test user's memories
    azure_results = mem0_search_client.search(
        search_text="*",
        filter=f"user_id eq '{test_user}'",
        include_total_count=True
    )

    print(f"   Documents found in Azure: {azure_results.get_count()}")
    for doc in azure_results:
        payload = doc.get('payload', {})
        if isinstance(payload, str):
            import json
            try:
                payload = json.loads(payload)
            except:
                pass
        print(f"   - Memory: {payload}")

except Exception as e:
    print(f"   Error: {e}")

print("\n5. Testing Mem0 version...")
# Check if we need to use a different method or property
if hasattr(memory, '__version__'):
    print(f"   Mem0 version: {memory.__version__}")
if hasattr(memory, 'version'):
    print(f"   Mem0 version: {memory.version}")

# Check available methods
print("\n6. Available memory methods:")
for attr in dir(memory):
    if not attr.startswith('_') and callable(getattr(memory, attr)):
        print(f"   - {attr}")

### สถานการณ์ที่ 2: การกลับมาเยี่ยม - วันหยุดพักผ่อนของครอบครัว (หลายสัปดาห์ต่อมา)


In [ ]:
print("\n\n🎯 SCENARIO 2: Sarah Returns Weeks Later for Family Vacation\n")
print("📅 Simulating time passing... Sarah starts a new conversation\n")

# Create a new thread to simulate a new conversation
sarah_thread_new = ChatHistoryAgentThread()

user_message3 = "Hi, my husband and I are planning another trip. We are looking for a good hotel!"

display_message("Sarah", user_message3, "#4fc3f7", "👤")

response3_content = ""
memories_retrieved = []

async for response in travel_agent.invoke(
    messages=user_message3,
    thread=sarah_thread_new
):
    if response.content:
        response3_content = str(response.content)

    if hasattr(response, 'thread'):
        sarah_thread_new = response.thread

        # Parse function calls
        async for msg in sarah_thread_new.get_messages():
            if hasattr(msg, 'items') and msg.items:
                for item in msg.items:
                    if hasattr(item, 'function_invoke_attempt') and item.function_invoke_attempt:
                        func_call = item.function_invoke_attempt

                        # Check for memory retrieval
                        if 'get_user_preferences' in func_call.function_name and hasattr(item, 'function_result'):
                            result = item.function_result.value
                            if result and "User preferences" in result:
                                display_memory_operation(
                                    "Retrieval", f"Found memories for {sarah_user_id}:\n{result}")

                        # Check for new preference storage
                        elif 'store_user_preference' in func_call.function_name:
                            display_memory_operation(
                                "Storage", f"Storing: {func_call.arguments.get('preference', '')}")

                        # Check for hotel search
                        elif 'search_hotels' in func_call.function_name:
                            display_function_call(
                                func_call.function_name,
                                func_call.arguments,
                                item.function_result.value if hasattr(
                                    item, 'function_result') else None
                            )

display_message("Travel Assistant", response3_content, "#81c784", "🤖")

In [ ]:
# Follow-up question
user_message4 = "Great suggestions! For the Maui option, what activities would you recommend for the kids?"

display_message("Sarah", user_message4, "#4fc3f7", "👤")

response4_content = ""
async for response in travel_agent.invoke(
    messages=user_message4,
    thread=sarah_thread_new
):
    if response.content:
        response4_content = str(response.content)
    sarah_thread_new = response.thread

display_message("Travel Assistant", response4_content, "#81c784", "🤖")

In [ ]:
print("\n🧪 TESTING MEMORY RETRIEVAL\n")

# First, ensure Sarah has some memories
test_preference = "I love romantic destinations with spa services"
result = travel_plugin.store_user_preference(sarah_user_id, test_preference)
print(f"Store result: {result}")

# Now test retrieval
preferences = travel_plugin.get_user_preferences(sarah_user_id)
print(f"\nRetrieved preferences:\n{preferences}")

# Also test the memory object directly
direct_memories = memory.get_all(user_id=sarah_user_id)
print(f"\nDirect memory.get_all() returned {len(direct_memories)} memories")
for i, mem in enumerate(direct_memories):
    print(f"Memory {i}: {mem}")

In [ ]:

# Check the Mem0 index structure in Azure AI Search
print("\n🔍 CHECKING MEM0 INDEX STRUCTURE\n")

try:
    # Get the mem0 index
    mem0_index = index_client.get_index("mem0")
    print("Mem0 index fields:")
    for field in mem0_index.fields:
        print(f"  - {field.name}: {field.type}")

    # Query the index directly
    mem0_search_client = SearchClient(
        endpoint=search_service_endpoint,
        index_name="mem0",
        credential=AzureKeyCredential(search_api_key)
    )

    # Get all documents for Sarah
    results = mem0_search_client.search(
        search_text="*",
        filter=f"user_id eq '{sarah_user_id}'",
        include_total_count=True
    )

    print(f"\nDocuments for {sarah_user_id}: {results.get_count()}")
    for doc in results:
        print(f"\nDocument ID: {doc.get('id')}")
        for key, value in doc.items():
            if key != 'id':
                print(
                    f"  {key}: {value[:100] if isinstance(value, str) and len(value) > 100 else value}")

except Exception as e:
    print(f"Error checking index: {str(e)}")

## สาธิตการค้นหาความจำเชิงความหมาย

พลังของ Mem0 มาจากการค้นหาเชิงความหมาย - การค้นหาความทรงจำที่เกี่ยวข้องโดยอิงจากความหมาย ไม่ใช่แค่คำสำคัญเท่านั้น


In [ ]:
print("🔍 SEMANTIC MEMORY SEARCH DEMONSTRATION\n")

# Search Sarah's memories for dietary-related information
dietary_search = memory.search(
    "dietary food allergies restrictions", user_id=sarah_user_id)

# Handle the dict response with 'results' key
if isinstance(dietary_search, dict) and 'results' in dietary_search:
    dietary_results = dietary_search.get('results', [])
else:
    dietary_results = dietary_search if isinstance(
        dietary_search, list) else []

print("Search Query: 'dietary food allergies restrictions'")
print(f"Results for Sarah:")
print("=" * 50)
if dietary_results:
    for mem in dietary_results:
        if isinstance(mem, dict):
            print(f"- {mem.get('memory', 'Unknown')}")
            print(f"  Relevance Score: {mem.get('score', 'N/A')}")
        else:
            print(f"- {mem}")
else:
    print("- No memories found")



## ประเด็นสำคัญ

### 1. หน่วยความจำผู้ใช้ที่คงอยู่
- **ความคงอยู่ข้ามเซสชัน**: การตั้งค่าของผู้ใช้จะถูกเก็บรักษาไว้แม้ในบทสนทนาที่ต่างกัน
- **การแยกผู้ใช้**: ผู้ใช้แต่ละคนมีพื้นที่หน่วยความจำของตัวเอง
- **บริบทอัตโนมัติ**: ตัวแทนจะดึงข้อมูลที่เกี่ยวข้องจากหน่วยความจำโดยอัตโนมัติ

### 2. ประโยชน์ของ Mem0
- **ความเข้าใจเชิงความหมาย**: ดึงข้อมูลจากหน่วยความจำโดยอิงจากความหมาย ไม่ใช่การจับคู่แบบตรงตัว
- **ความสามารถในการขยายตัว**: ใช้ Azure AI Search สำหรับการจัดเก็บข้อมูลระดับองค์กร
- **ความเป็นส่วนตัว**: หน่วยความจำของผู้ใช้ถูกแยกและปลอดภัย

### 3. ประสบการณ์ผู้ใช้ที่ดียิ่งขึ้น
- **ไม่มีการซ้ำซ้อน**: ผู้ใช้ไม่จำเป็นต้องบอกการตั้งค่าซ้ำ
- **การปรับให้เหมาะกับแต่ละบุคคล**: คำแนะนำดีขึ้นเมื่อเวลาผ่านไป
- **การรับรู้บริบท**: ตัวแทนเข้าใจประวัติของผู้ใช้


## สรุป

ยินดีด้วย! คุณได้สร้างตัวแทนการท่องเที่ยว AI ที่มีความสามารถในการจดจำข้อมูลอย่างต่อเนื่องสำเร็จ โดยใช้:

- **Mem0**: สำหรับการจัดการหน่วยความจำที่ชาญฉลาดและต่อเนื่อง
- **Azure AI Search**: เป็นแหล่งจัดเก็บเวกเตอร์ที่สามารถขยายได้สำหรับข้อมูลหน่วยความจำและการเดินทาง
- **Semantic Kernel**: เพื่อจัดการตัวแทนและปลั๊กอินต่างๆ

## สิ่งที่คุณได้เรียนรู้:
1. วิธีการรวม Mem0 กับ Azure AI Search เพื่อการจดจำข้อมูลอย่างต่อเนื่อง
2. การสร้างปลั๊กอิน Semantic Kernel ที่ใช้ประโยชน์จากหน่วยความจำ
3. การสร้างตัวแทนที่สามารถจดจำความชอบของผู้ใช้ในแต่ละเซสชัน
4. การใช้การค้นหาเชิงความหมายเพื่อดึงข้อมูลหน่วยความจำที่เกี่ยวข้อง

## การใช้งานในโลกจริง:
- **บริการลูกค้า**: จดจำประวัติและความชอบของลูกค้า
- **ผู้ช่วยส่วนตัว**: รักษาบริบทการใช้งานในช่วงเวลาหลายวันหรือหลายสัปดาห์
- **การดูแลสุขภาพ**: ติดตามข้อมูลและความชอบของผู้ป่วย
- **การศึกษา**: จดจำความก้าวหน้าและรูปแบบการเรียนรู้ของนักเรียน
- **อีคอมเมิร์ซ**: การช้อปปิ้งแบบเฉพาะบุคคลตามประวัติการใช้งาน

## ขั้นตอนถัดไป:
- เพิ่มการหมดอายุของหน่วยความจำสำหรับข้อมูลที่มีความสำคัญตามเวลา
- เพิ่มการให้คะแนนความสำคัญของหน่วยความจำ
- สร้างระบบตัวแทนหลายตัวที่มีหน่วยความจำร่วมกัน
- รวมเข้ากับระบบ CRM สำหรับการใช้งานในองค์กร
- เพิ่มการจัดการเวอร์ชันของหน่วยความจำและการตรวจสอบย้อนหลัง



---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามให้การแปลมีความถูกต้องมากที่สุด แต่โปรดทราบว่าการแปลโดยอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่ถูกต้อง เอกสารต้นฉบับในภาษาดั้งเดิมควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ ขอแนะนำให้ใช้บริการแปลภาษามนุษย์ที่มีความเชี่ยวชาญ เราไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความผิดที่เกิดจากการใช้การแปลนี้
